In [1]:
import pipeline
from PIL import Image
import torch
from model_loader import load_models
import matplotlib.pyplot as plt
import numpy as np
from ddpm import DDPMSampler
from tqdm import tqdm
import torchvision
from Dataset import PineappleDataset
import os

In [2]:
def get_time_embedding(timesteps: torch.LongTensor, dim: int = 160):
    if timesteps.dim() == 0:
        timesteps = timesteps.unsqueeze(0)
    device = timesteps.device
    half_dim = dim
    freqs = torch.pow(
        10000,
        -torch.arange(0, half_dim, dtype=torch.float32, device=device) / half_dim
    )
    args = timesteps.float().unsqueeze(1) * freqs.unsqueeze(0)
    emb = torch.cat([torch.cos(args), torch.sin(args)], dim=-1)
    return emb
def sample_i(h, w, vae, diffusion_model, generator, device,sigma_latent,num_image,path_to_save):
    sampler_i = DDPMSampler(generator)
    sampler_i.set_inference_timesteps(50)
    with torch.no_grad():
        latents = torch.randn((1, 4, h // 8, w // 8), device=device)
        for timestep in tqdm(sampler_i.timesteps, desc="Sampling"):
            t = torch.tensor([int(timestep)], dtype=torch.long, device=device)
            time_embedding = get_time_embedding(t).to(device)
            model_output = diffusion_model(latents, time_embedding)
            latents = sampler_i.step(timestep, latents, model_output)
        
        latents = latents*sigma_latent
        # Decode & log
        decoded = vae.decoder(latents)
        img = decoded.squeeze(0).cpu().numpy()
        img = np.transpose(img, (1, 2, 0))
        # BGR → RGB
        img = img[..., ::-1] 
        img = np.clip(img, 0.0, 1.0)
        img = (img * 255).astype(np.uint8)
        # convert to float in [0,1]
        tensor = torch.from_numpy(img).permute(2,0,1).float().div(255.0)
        torchvision.utils.save_image(tensor, os.path.join(path_to_save,f"outputSCALED_{num_image}.png"))
    #return img

In [ ]:
from model_loader_conv import load_models
DEVICE = "cpu"

ALLOW_CUDA = True

if torch.cuda.is_available() and ALLOW_CUDA:
    DEVICE = "cuda"
diffusion,vae = load_models("checkpoints\diffusion/betaKL@1.0_conv_NU_constantLR_scaled/diffusion_best_epoch735_loss0.3347_1.pt",
                             "checkpoints/vae/betaKL@1.0/weights_ck_398.pt",
                             DEVICE
                             )
vae.eval()
diffusion.eval()
seed = 15
base_path = "../betaKL@1.0_conv_NU_constantLR_scaled/epoch735_2"
base_path = os.path.join(base_path, f"seed_{seed}")
# Create the directory if it doesn't exist
if not os.path.exists(base_path):
    os.makedirs(base_path)
img_list = []
generator = torch.Generator(device=DEVICE)
generator.manual_seed(seed)
for image_num in range(982):
    sample_i(256, 256, vae, diffusion, generator, DEVICE, 0.182473480701,image_num,base_path)
    #img_list.append(img_numpy)

Sampling: 100%|██████████| 50/50 [00:02<00:00, 16.75it/s]

In [3]:
from model_loader_conv import load_models
DEVICE = "cpu"

ALLOW_CUDA = True

if torch.cuda.is_available() and ALLOW_CUDA:
    DEVICE = "cuda"
diffusion,vae = load_models("checkpoints/diffusion/betaKL@1.0_conv_U_decayLR_scaled/diffusion_best_epoch654_loss0.3315_1.pt",
                             "checkpoints/vae/betaKL@1.0/weights_ck_398.pt",
                             DEVICE
                             )
vae.eval()
diffusion.eval()
seed = 15
img_list = []
generator = torch.Generator(device=DEVICE)
generator.manual_seed(seed)

base_path = "../betaKL@1.0_conv_U_decayLR_scaled/epoch654"
base_path = os.path.join(base_path, f"seed_{seed}")
# Create the directory if it doesn't exist
if not os.path.exists(base_path):
    os.makedirs(base_path)
for image_num in range(982):
    sample_i(256, 256, vae, diffusion, generator, DEVICE, 0.182692706585,image_num,base_path)

Sampling: 100%|██████████| 50/50 [00:02<00:00, 17.58it/s]


In [ ]:
dataset = PineappleDataset(train=False, train_ratio=0.8, dataset_path="C:/Users/manfr/OneDrive/Escritorio/Manfred/EMAI/master_research/pineapples_diffusion/FULL_UNIFIED/*")
# print the len
print(len(dataset))

982
